### Lesson 3

In [ ]:
import os
import sys
import langchain
import subprocess
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from Update_Git import git_add, git_commit, git_push
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_dir, ".env")
load_dotenv(dotenv_path=env_path)

add_file = os.path.join(current_dir, 'Lesson_3.ipynb')
git_add(add_file)
git_commit('Lesson 2 updated')
git_push('main')

''

In [ ]:
llm = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature=0.5
)

def call_string_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI chef. Give me 3 synonyms for the following word."),
        ("human", "{input}")
    ])

    parser = StrOutputParser()

    chain = prompt | llm | parser

    # The output of llm is a AI message, se we need to convert it to a string
    # The outout of the parser is a string

    return chain.invoke("nice")
                        
print(call_string_output_parser())


1. Pleasant
2. Agreeable
3. Kind


In [9]:
def call_list_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Give me 3 synonyms for the following word, separated by commas."),
        ("human", "{input}")
    ])

    parser = CommaSeparatedListOutputParser()

    chain = prompt | llm | parser

    return chain.invoke("morning")

print(call_list_output_parser())

['dawn', 'forenoon', 'sunrise']


In [ ]:
def call_json_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract information from the following phrase.\n Formatting instructuions: {format_instructions}"),
        ("human", "{input}")
    ])

    parser = CommaSeparatedListOutputParser()

    chain = prompt | llm | parser

    return chain.invoke("morning")